In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as ppt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pickle

pd.set_option('display.max_columns', 50)

In [2]:
df = pd.read_csv('train_keystroke.csv')

In [3]:
df.describe()

,user,press-0,release-0,press-1,release-1,press-2,release-2,press-3,release-3,press-4,release-4,press-5,release-5,press-6,release-6,press-7,release-7,press-8,release-8,press-9,release-9,press-10,release-10,press-11,release-11,press-12,release-12
count,880.000000,880.0,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000
mean,55.500000,0.0,69.740909,192.504545,280.730682,378.078409,460.992045,625.425000,719.445455,792.982955,878.523864,1043.360227,1131.386364,1242.047727,1327.745455,1485.427273,1575.250000,1683.110227,1773.718182,1874.559091,1973.450000,2093.876136,2194.747727,2272.895455,2366.973864,2491.186364,2587.061364
std,31.771009,0.0,35.830579,118.281523,128.337586,243.485675,246.432639,359.327185,362.141729,403.595652,405.840245,465.689141,467.088020,525.164691,527.872734,605.835537,607.896069,654.999470,657.806567,724.772838,725.310795,793.458930,792.913889,856.416953,856.341509,905.936922,905.291522
min,1.000000,0.0,0.000000,0.000000,1.000000,1.000000,1.000000,2.000000,4.000000,14.000000,83.000000,221.000000,318.000000,388.000000,470.000000,562.000000,648.000000,667.000000,763.000000,793.000000,924.000000,891.000000,1028.000000,955.000000,1055.000000,1124.000000,1188.000000
25%,28.000000,0.0,50.000000,152.000000,232.000000,281.000000,374.000000,463.000000,561.000000,592.750000,676.000000,795.500000,889.500000,950.750000,1037.000000,1130.000000,1222.500000,1288.500000,1375.000000,1433.500000,1532.500000,1589.750000,1708.000000,1741.750000,1834.250000,1930.500000,2034.500000
50%,55.500000,0.0,72.000000,192.000000,279.000000,358.000000,436.000000,569.000000,663.000000,721.500000,807.000000,944.000000,1034.000000,1131.500000,1214.500000,1346.500000,1432.000000,1526.000000,1618.500000,1695.000000,1791.500000,1896.000000,1994.000000,2059.500000,2151.000000,2255.500000,2344.000000
75%,83.000000,0.0,94.000000,231.250000,328.000000,424.000000,501.250000,706.500000,800.000000,888.500000,978.500000,1161.500000,1247.750000,1391.250000,1471.250000,1688.250000,1788.000000,1921.250000,2004.000000,2145.000000,2256.000000,2390.250000,2474.500000,2585.750000,2687.250000,2823.250000,2905.750000
max,110.000000,0.0,241.000000,1287.000000,1343.000000,4333.000000,4453.000000,4823.000000,4890.000000,5191.000000,5263.000000,6239.000000,6377.000000,6497.000000,6624.000000,7039.000000,7136.000000,7377.000000,7471.000000,7663.000000,7753.000000,8006.000000,8063.000000,8189.000000,8259.000000,8422.000000,8487.000000


from describe(): it appears that one/many of the 880 rows only wrote 12 characters not 13, and that came from 'press-0' 'release-0' 'press-1'.

as HT is 0 for the first character and started writing from second character.

In [4]:
df[(df['press-0'] == 0) & (df['release-0'] == 0)] # 33 rows

,user,press-0,release-0,press-1,release-1,press-2,release-2,press-3,release-3,press-4,release-4,press-5,release-5,press-6,release-6,press-7,release-7,press-8,release-8,press-9,release-9,press-10,release-10,press-11,release-11,press-12,release-12
33,5,0,0,41,185,209,297,418,489,585,657,705,753,967,1017,1129,1225,1297,1345,1465,1561,1649,1697,1769,1865,2047,2114
34,5,0,0,0,78,127,198,295,366,455,550,672,766,839,886,1064,1165,1191,1238,1310,1430,1503,1574,1622,1742,1855,1974
111,14,0,0,7,100,264,359,566,636,781,877,1046,1109,1165,1260,1469,1570,1637,1733,1830,1917,1969,2111,2181,2277,2414,2509
117,15,0,0,129,184,315,385,455,527,672,745,861,942,1001,1047,1095,1167,1315,1385,1551,1615,1811,1879,2026,2125,2257,2309
147,19,0,0,79,99,139,235,379,475,499,598,1222,1286,1404,1549,1731,1810,1905,1999,2085,2170,2251,2371,2422,2515,2661,2731
148,19,0,0,0,1,1,8,143,240,290,390,496,571,691,786,954,1026,1151,1207,1303,1380,1471,1592,1662,1726,1854,1944
155,20,0,0,51,121,217,289,3077,3226,3153,3249,3392,3465,3614,3714,4553,4625,5361,5433,5865,5945,6265,6413,6351,6483,6601,6689
173,22,0,0,0,1,1,49,145,233,245,338,501,527,605,641,690,802,897,995,1046,1139,1239,1377,1370,1483,1545,1634
174,22,0,0,98,152,349,392,462,558,535,630,749,838,914,960,1022,1137,1249,1318,1358,1455,1551,1694,1667,1745,1862,1958
175,22,0,0,99,137,320,387,858,956,1003,1106,1230,1285,1355,1412,1453,1523,1643,1714,1778,1874,1970,2154,2144,2198,2297,2385


33 records only wrote 12 character
also noticed 'press-X' has very low values and 'release-X' also ( values like V < 10), assuming this numbers are in millisecond, that comes to conclude that this isn't a human (robot/script) or copy-pasted those characters 

as the model is focused to select which user from test data, and not indentify if this user is human or robot.
then we can replace those low values in columns ( X < 1st quartile (25%) or a certain value), with mean/mode value for the same column. to complete our data

In [5]:
df[(df['press-1'] <= 20) & (df['press-2'] <= 30)] #  24 rows

,user,press-0,release-0,press-1,release-1,press-2,release-2,press-3,release-3,press-4,release-4,press-5,release-5,press-6,release-6,press-7,release-7,press-8,release-8,press-9,release-9,press-10,release-10,press-11,release-11,press-12,release-12
134,17,0,3,1,3,3,19,402,570,478,611,776,915,1022,1090,1179,1275,1347,1541,1435,1555,1651,1844,1748,1915,1844,2036
146,19,0,1,1,1,1,102,211,307,339,459,570,669,819,885,1140,1221,1337,1476,1522,1599,1709,1802,1881,2007,2118,2210
148,19,0,0,0,1,1,8,143,240,290,390,496,571,691,786,954,1026,1151,1207,1303,1380,1471,1592,1662,1726,1854,1944
151,19,0,1,1,1,2,2,2,4,14,83,221,318,434,532,700,804,882,978,1071,1170,1275,1364,1410,1505,1644,1737
173,22,0,0,0,1,1,49,145,233,245,338,501,527,605,641,690,802,897,995,1046,1139,1239,1377,1370,1483,1545,1634
257,33,0,4,4,4,15,117,280,303,345,391,546,614,631,737,824,919,919,992,1085,1187,1199,1319,1346,1447,1560,1685
260,33,0,1,1,1,1,2,86,159,253,349,589,686,780,859,1225,1294,1389,1461,1582,1701,2389,2461,2517,2637,2717,2819
262,33,0,1,1,1,23,110,230,316,351,398,548,624,625,710,798,901,918,990,1066,1182,1159,1255,1326,1398,1541,1678
263,33,0,1,1,1,2,2,2,52,132,174,299,388,388,470,572,668,702,770,836,967,968,1051,1079,1149,1284,1387
332,42,0,0,1,1,1,33,106,177,326,394,513,596,627,746,794,865,1040,1105,1257,1341,1529,1606,1748,1841,1989,2062


data needs to be treated carefully knowing it isn't clean data, and it will be clear what to do when we extract our features

# 1st Step - creating four new new features

In [6]:
df[f'hold time-00'] = df[f'release-0'] - df[f'press-0']
for i in range(1, 13):
    df[f'hold time-{i:02}'] = df[f'release-{i}'] - df[f'press-{i}']
    df[f'ppt-{i:02}'] = df[f'press-{i}'] - df[f'press-{i - 1}']
    df[f'rrt-{i:02}'] = df[f'release-{i}'] - df[f'release-{i - 1}']
    df[f'rpt-{i:02}'] = df[f'press-{i}'] - df[f'release-{i - 1}']
feature_cols = sorted(list(df.columns[27:]))
feature_cols.append('user')
feature_cols

['hold time-00',
 'hold time-01',
 'hold time-02',
 'hold time-03',
 'hold time-04',
 'hold time-05',
 'hold time-06',
 'hold time-07',
 'hold time-08',
 'hold time-09',
 'hold time-10',
 'hold time-11',
 'hold time-12',
 'ppt-01',
 'ppt-02',
 'ppt-03',
 'ppt-04',
 'ppt-05',
 'ppt-06',
 'ppt-07',
 'ppt-08',
 'ppt-09',
 'ppt-10',
 'ppt-11',
 'ppt-12',
 'rpt-01',
 'rpt-02',
 'rpt-03',
 'rpt-04',
 'rpt-05',
 'rpt-06',
 'rpt-07',
 'rpt-08',
 'rpt-09',
 'rpt-10',
 'rpt-11',
 'rpt-12',
 'rrt-01',
 'rrt-02',
 'rrt-03',
 'rrt-04',
 'rrt-05',
 'rrt-06',
 'rrt-07',
 'rrt-08',
 'rrt-09',
 'rrt-10',
 'rrt-11',
 'rrt-12',
 'user']

now df has (13 hold time - 12 ppt - 12 rrt - 12 rpt) new columns
and lets create a new df which has those new columns and analysis what we have nw

In [7]:
df

,user,press-0,release-0,press-1,release-1,press-2,release-2,press-3,release-3,press-4,release-4,press-5,release-5,press-6,release-6,press-7,release-7,press-8,release-8,press-9,release-9,press-10,release-10,press-11,release-11,...,rpt-06,hold time-07,ppt-07,rrt-07,rpt-07,hold time-08,ppt-08,rrt-08,rpt-08,hold time-09,ppt-09,rrt-09,rpt-09,hold time-10,ppt-10,rrt-10,rpt-10,hold time-11,ppt-11,rrt-11,rpt-11,hold time-12,ppt-12,rrt-12,rpt-12
0,1,0,120,216,312,424,496,592,664,808,856,1000,1072,1304,1400,1496,1544,1712,1760,1992,2064,2376,2448,2584,2632,...,232,48,192,144,96,48,216,216,168,72,280,304,232,72,384,384,312,48,208,184,136,72,168,192,120
1,1,0,95,168,265,360,455,527,599,736,807,928,999,1024,1095,1215,1271,1423,1471,1664,1711,1880,1952,2039,2111,...,25,56,191,176,120,48,208,200,152,47,241,240,193,72,216,241,169,72,159,159,87,48,192,168,120
2,1,0,71,143,231,783,903,1087,1159,1351,1454,1559,1631,1703,1799,1823,1902,2039,2111,2271,2343,2487,2559,2679,2751,...,72,79,120,103,24,72,216,209,137,72,232,232,160,72,216,216,144,72,192,192,120,55,192,175,120
3,1,0,95,144,263,353,431,760,832,1159,1207,1327,1377,1500,1591,2968,3015,3151,3223,3415,3463,3631,3703,3815,3887,...,123,47,1468,1424,1377,72,183,208,136,48,264,240,192,72,216,240,168,72,184,184,112,72,168,168,96
4,1,0,70,166,238,310,406,526,598,710,758,878,950,926,1022,1094,1166,1310,1382,1543,1605,1734,1806,1926,1998,...,-24,72,168,144,72,72,216,216,144,62,233,223,161,72,191,201,129,72,192,192,120,96,160,184,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,110,0,80,225,306,444,465,575,672,689,784,884,1040,1000,1123,1439,1576,1638,1735,1785,1884,1928,2040,2057,2186,...,-40,137,439,453,316,97,199,159,62,99,147,149,50,112,143,156,44,129,129,146,17,220,227,318,98
876,110,0,81,217,289,387,473,954,1057,1091,1189,1265,1419,1384,1465,1595,1698,1755,1866,1921,2019,2161,2265,2351,2457,...,-35,103,211,233,130,111,160,168,57,98,166,153,55,104,240,246,142,106,190,192,86,143,210,247,104
877,110,0,79,216,300,393,519,692,807,807,894,1004,1204,1161,1279,1407,1512,1591,1724,1763,1866,1879,2046,2048,2192,...,-43,105,246,233,128,133,184,212,79,103,172,142,39,167,116,180,13,144,169,146,2,146,255,257,111
878,110,0,84,200,290,384,476,782,930,943,978,1064,1240,1200,1310,1413,1484,1537,1658,1760,1848,1831,1940,1944,2082,...,-40,71,213,174,103,121,124,174,53,88,223,190,102,109,71,92,-17,138,113,142,4,145,230,237,92


In [8]:
ndf = df[feature_cols].copy()
ndf.reset_index(inplace=True)

In [9]:
ndf.set_index('index', inplace=True)

In [10]:
ndf.describe()

,hold time-00,hold time-01,hold time-02,hold time-03,hold time-04,hold time-05,hold time-06,hold time-07,hold time-08,hold time-09,hold time-10,hold time-11,hold time-12,ppt-01,ppt-02,ppt-03,ppt-04,ppt-05,ppt-06,ppt-07,ppt-08,ppt-09,ppt-10,ppt-11,ppt-12,rpt-01,rpt-02,rpt-03,rpt-04,rpt-05,rpt-06,rpt-07,rpt-08,rpt-09,rpt-10,rpt-11,rpt-12,rrt-01,rrt-02,rrt-03,rrt-04,rrt-05,rrt-06,rrt-07,rrt-08,rrt-09,rrt-10,rrt-11,rrt-12,user
count,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.000000,880.00000,880.000000
mean,69.740909,88.226136,82.913636,94.020455,85.540909,88.026136,85.697727,89.822727,90.607955,98.890909,100.871591,94.078409,95.875000,192.504545,185.573864,247.346591,167.557955,250.377273,198.687500,243.379545,197.682955,191.448864,219.317045,179.019318,218.290909,122.763636,97.347727,164.432955,73.537500,164.836364,110.661364,157.681818,107.860227,100.840909,120.426136,78.147727,124.212500,210.989773,180.261364,258.453409,159.078409,252.862500,196.359091,247.504545,198.468182,199.731818,221.297727,172.226136,220.08750,55.500000
std,35.830579,39.349084,31.485219,36.921767,30.417956,30.164517,31.290914,30.160706,38.364721,36.269305,43.352964,41.590008,33.667082,118.281523,194.601764,222.058021,119.499464,157.805531,159.435206,204.335014,106.241426,169.253180,181.904564,144.299929,123.585188,112.888494,196.557997,222.402351,125.062209,155.331480,160.423774,204.612803,107.478568,170.181920,185.218486,149.917500,131.277339,118.550992,194.692581,223.265658,124.146330,156.101606,159.729758,205.464246,111.700585,169.046805,184.965009,146.403878,126.41496,31.771009
min,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,0.000000,1.000000,15.000000,0.000000,0.000000,0.000000,4.000000,7.000000,23.000000,30.000000,16.000000,24.000000,24.000000,1.000000,13.000000,-117.000000,-97.000000,-89.000000,-119.000000,-98.000000,-71.000000,-94.000000,-120.000000,-256.000000,-122.000000,-164.000000,-237.000000,0.000000,0.000000,2.000000,0.000000,25.000000,3.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.00000,1.000000
25%,50.000000,68.000000,67.000000,71.000000,68.000000,70.000000,66.750000,71.000000,68.000000,72.000000,72.000000,70.000000,72.000000,152.000000,125.000000,147.000000,97.000000,185.750000,126.000000,144.000000,142.000000,118.000000,136.000000,115.000000,169.000000,84.750000,22.000000,69.000000,-1.000000,107.000000,38.000000,61.750000,54.750000,24.000000,40.000000,1.000000,88.000000,166.000000,119.000000,164.750000,76.000000,192.000000,122.000000,144.000000,142.000000,128.000000,134.000000,96.000000,176.00000,28.000000
50%,72.000000,82.000000,76.000000,91.500000,82.000000,87.500000,81.000000,90.000000,83.000000,96.000000,94.500000,88.000000,95.000000,192.000000,168.000000,216.000000,145.000000,216.000000,166.000000,181.500000,185.000000,155.000000,180.000000,152.000000,202.000000,121.000000,88.000000,135.500000,64.000000,135.000000,76.000000,101.000000,96.000000,71.000000,86.500000,64.000000,120.000000,208.000000,168.000000,228.500000,144.000000,220.000000,164.500000,192.000000,187.000000,168.000000,190.000000,149.500000,210.00000,55.500000
75%,94.000000,103.250000,97.000000,118.000000,100.000000,105.000000,102.000000,105.000000,110.000000,120.000000,124.000000,111.000000,116.000000,231.250000,203.000000,289.250000,204.500000,264.250000,210.000000,263.250000,228.250000,208.000000,240.000000,208.000000,240.000000,152.250000,130.000000,209.000000,120.000000,169.000000,123.250000,174.750000,142.000000,119.000000,144.000000,115.00

from only min for hold time, you can see that a lot of characters have been automated ( written by script or copy pasted)

also noticed, negative values in rpt-X, which could be typing fast where pressing is faster than releasing. (release > press), and couldn't neglect the sign because this will change the data and produced inaccurate data.

# 2nd Step - creating our training features

calculating the mean and std for each one of 4 main features and from that a new data frame which will have the final 8 features and user class

In [11]:
ndf['hold time-mean'] = ndf[feature_cols[:13]].mean(axis=1)
ndf['ppt-mean'] = ndf[feature_cols[13:25]].mean(axis=1)
ndf['rpt-mean'] = ndf[feature_cols[25:37]].mean(axis=1)
ndf['rrt-mean'] = ndf[feature_cols[37:49]].mean(axis=1)
ndf['hold time-std'] = ndf[feature_cols[:13]].std(axis=1)
ndf['ppt-std'] = ndf[feature_cols[13:25]].std(axis=1)
ndf['rpt-std'] = ndf[feature_cols[25:37]].std(axis=1)
ndf['rrt-std'] = ndf[feature_cols[37:49]].std(axis=1)

In [12]:
ndf[['hold time-mean', 'ppt-mean', 'rpt-mean', 'rrt-mean', 'hold time-std', 'ppt-std', 'rpt-std', 'rrt-std', 'user']]

,hold time-mean,ppt-mean,rpt-mean,rrt-mean,hold time-std,ppt-std,rpt-std,rrt-std,user
index,,,,,,,,,
0,72.000000,229.333333,157.333333,225.333333,21.908902,63.115094,67.910815,73.069498,1
1,70.384615,185.916667,113.666667,182.000000,17.523976,36.412931,46.529235,40.174619,1
2,80.307692,239.250000,156.833333,237.916667,17.172578,136.361235,133.531633,145.322499,1
3,72.000000,331.916667,259.916667,330.000000,21.205345,368.330498,362.565720,353.436377,1
4,74.769231,173.833333,100.833333,176.000000,13.869354,47.510445,47.613660,39.874804,1
...,...,...,...,...,...,...,...,...,...
875,111.307692,190.333333,88.083333,202.000000,46.370581,89.469480,89.940848,101.799268,110
876,103.076923,213.416667,113.666667,218.583333,23.350452,116.619395,125.816124,128.481664,110
877,123.615385,191.916667,70.166667,197.500000,34.767174,54.841024,66.644011,73.933016,110


assigning features to variable X and user class to variable y and splitting the data to train dataand test data

In [22]:
X = ndf[['hold time-mean', 'ppt-mean', 'rpt-mean', 'rrt-mean', 'hold time-std', 'ppt-std', 'rpt-std', 'rrt-std']].copy()
y = df['user'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 3rd Step - Training machine learning models

creating pipelines for randomforest, support vector machine, and xgboost. As specified, no need for feature selection, cross valudation or parameters tuning, the classifier is using the default values.

In [43]:
rf = Pipeline([
    ('cls', RandomForestClassifier())
    ])
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.19886363636363635

In [44]:
svm = Pipeline([
    ('cls', SVC())
])
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.028409090909090908

In [45]:
xgb = Pipeline([
    ('cls', XGBClassifier())
])

xgb.fit(X_train, y_train)
xgb.score(X_test, y_test)

C:\Users\Muhammad\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:41:36] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.21022727272727273

In [42]:
pickle.dump(rf, open('rf.model', 'wb'))
pickle.dump(svm, open('svm.model', 'wb'))
pickle.dump(xgb, open('xgb.model', 'wb'))